In [1]:
from utils import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import h5py

In [2]:
f = h5py.File(os.path.join(RECURSION_DIR, 'h5', 'train-controls-f32.h5'), 'r')

In [16]:
dataset_x, dataset_y = f['dataset_x'], f['dataset_y']

In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(6, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, 3),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, 3),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.block5 = nn.Sequential(
            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  
        )
        self.drop_out = nn.Dropout()
        
        # Linear layers
        self.fc1 = nn.Linear(2048, 29)
        self.fc2 = nn.Linear(29, 29)
#         self.fc3 = nn.Linear(2000, 1108)

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [56]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset_x, dataset_y):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        self.dataset_x = dataset_x
        self.dataset_y = dataset_y

    def __len__(self):
        return len(self.dataset_x)

    def __getitem__(self, idx):
        x = torch.Tensor(self.dataset_x[idx])
        x = x.T
        y = torch.Tensor([self.dataset_y[idx]])
        
        sample = {'image': x,  'label': y}
        return sample

In [57]:
def load_data(dataset_x, dataset_y, batch_size=32):
    train_set = Dataset(dataset_x, dataset_y)
    return torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False)
    

def train(net, trainloader, save_path, num_epoch=1):
    '''
    Function: train
    arguments:
        net - CNN model used for training
        trainloader - Torch DataLoader object
        save_path - path to save the trained model
    '''
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters())
        
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net.to(device)
    
    for epoch in range(num_epoch):
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data['image'].to(device), data['label'].to(device, dtype=torch.long)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels.squeeze())
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if (i + 1) % 50 == 0:
                print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0
            
        print(f'epoch-{epoch+1} completed.')
            
    torch.save(net.state_dict(), save_path)

In [58]:
trainloader = load_data(dataset_x, dataset_y)

In [65]:
net = Net()

In [66]:
SAVE_PATH = os.path.join(os.getcwd(), 'kaan-models')

In [67]:
train(net, trainloader, SAVE_PATH, num_epoch=1)

[1,    50] loss: 3.378
[1,   100] loss: 3.370
[1,   150] loss: 3.378
[1,   200] loss: 3.372
epoch-1 completed.


In [46]:
len(list(net.parameters()))

30